In [ ]:
import { load } from "dotenv";
const env = await load();

const process = {
  env,
};


In [ ]:
import { ChatMoonshot } from "@langchain/community/chat_models/moonshot";
import { HumanMessage } from "@langchain/core/messages";

const chatModel = new ChatMoonshot({
  apiKey: process.env.OPENAI_API_KEY,
});

await chatModel.invoke([new HumanMessage("Tell me a joke")]);

In [ ]:
import { ChatMoonshot } from "@langchain/community/chat_models/moonshot";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatModel = new ChatMoonshot({
  apiKey: process.env.OPENAI_API_KEY,
});

const parser = new StringOutputParser();

const chian = chatModel.pipe(parser);

await chian.invoke([new HumanMessage("Tell me a joke")]);


In [ ]:
import { StructuredOutputParser} from '@langchain/core/output_parsers'

const parser = StructuredOutputParser.fromNamesAndDescriptions({
    answer:'用户问题的答案',
    evidence:'你回答用户问题所依据的答案',
    confidence:'问题答案的可信度评分，格式是百分数'
})

console.log(parser.getFormatInstructions());


In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";
import { StructuredOutputParser } from "@langchain/core/output_parsers";
import { ChatMoonshot } from "@langchain/community/chat_models/moonshot";

const chatModel = new ChatMoonshot({
  apiKey: process.env.OPENAI_API_KEY,
});

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});

const prompt = PromptTemplate.fromTemplate('尽可能的回答用的问题\n{instructions}\n{question}')

const chain = prompt.pipe(chatModel).pipe(parser);

const res = await chain.invoke({
    question:'蒙娜丽莎的作者是谁？是什么时候绘制的',
    instructions:parser.getFormatInstructions()
})

console.log(res);


In [ ]:
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";

const parser = new CommaSeparatedListOutputParser();

console.log(parser.getFormatInstructions());


In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatMoonshot } from "@langchain/community/chat_models/moonshot";

const chatModel = new ChatMoonshot({
  apiKey: process.env.OPENAI_API_KEY,
});

const parser = new CommaSeparatedListOutputParser();

const prompt = PromptTemplate.fromTemplate(
  "列出3个{country}著名的互联网公司。\n{instructions}"
);

const chain = prompt.pipe(chatModel).pipe(parser);

const res = await chain.invoke({
  country: "America",
  instructions: parser.getFormatInstructions(),
});

console.log(res);


In [ ]:
import { z } from "zod";
import {
  StructuredOutputParser,
  OutputFixingParser,
} from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";

const schema = z.object({
  answer: z.string().describe("用户问题的答案"),
  confidence: z
    .number()
    .min(0)
    .max(100)
    .describe("问题答案的可信度评分，满分 100"),
});


In [ ]:
const parser = StructuredOutputParser.fromZodSchema(schema);
const prompt = PromptTemplate.fromTemplate(
  "尽可能的回答用的问题 \n{instructions} \n{question}"
);
const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});
const chain = prompt.pipe(model).pipe(parser);
const res = await chain.invoke({
  question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
  instructions: parser.getFormatInstructions(),
});

console.log(res);


In [ ]:
const wrongOutput = {
  answer:
    "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  sources: "90%",
};

const fixParser = OutputFixingParser.fromLLM(model, parser);
const output = await fixParser.parse(JSON.stringify(wrongOutput));


In [ ]:
const wrongOutput = {
  answer:
    "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  sources: "-1",
};

const fixParser = OutputFixingParser.fromLLM(model, parser);
const output = await fixParser.parse(JSON.stringify(wrongOutput));

output;
